In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os


In [ ]:

IMAGE_FILES = []

folder = "C:/project/deployment/dataset"


In [ ]:

for celebrity in os.listdir(folder):
    for image in os.listdir(f"{folder}/{celebrity}"):
        if len(image.split(".")) > 1 and image.split(".")[-1] == 'jpg':
            IMAGE_FILES.append(celebrity + '/' + image)
print(len(IMAGE_FILES))
padding = [ 0.5, 0.1, 0.2, 0.2]

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

# For static images:
with mp_face_detection.FaceDetection(
    model_selection=1, min_detection_confidence=0.5) as face_detection:
  for idx, file in enumerate(IMAGE_FILES):
    image = cv2.imread(f"{folder}/{file}")

    if not np.any(image):
        continue
    # Convert the BGR image to RGB and process it with MediaPipe Face Detection.
    results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # Draw face detections of each face.
    if not results.detections:
      continue
    for detection in results.detections:
        box = detection.location_data.relative_bounding_box
        rect_coords = np.array(
            [
                [
                    box.xmin - box.height * padding[2],
                    box.ymin - box.height * padding[0],
                ],
                [
                    box.xmin + box.width * (1 + padding[3]),
                    box.ymin + box.height * (1 + padding[1]),
                ],
            ]
        )
        rect_coords[:, 1] *= image.shape[0]
        rect_coords[:, 0] *= image.shape[1]
        rect_coords = rect_coords.astype(int)
        face = image[
            max(rect_coords[0, 1], 0) : rect_coords[1, 1],
            max(rect_coords[0, 0], 0): rect_coords[1, 0],
        ]


        final_path = f'./latest_cropped/{file}'
        dir_path = os.path.dirname(final_path)

        if not os.path.exists(dir_path):
            os.makedirs(dir_path)

        face = cv2.resize(face, (128, 128))

        cv2.imwrite(final_path, face)